In [34]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from datetime import datetime

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [14]:
X=np.arange(0,2000).reshape(-1,4)
y=np.arange(5,(X.shape[0]+1)*5,5)

temp=pd.DataFrame(data=X)
temp['target']=pd.Series(y.reshape(-1,))
from sklearn.preprocessing import StandardScaler
inputs_standardizer=StandardScaler()
inputs_standardizer.fit_transform(temp.drop('target',axis=1).values.tolist())

target_standardizer=StandardScaler()
target_standardizer.fit_transform(temp['target'].values.reshape(-1,1).tolist())

norm_temp=pd.DataFrame(data=inputs_standardizer.fit_transform(temp.drop('target',axis=1).values.tolist()))
norm_temp['target']=pd.Series(target_standardizer.fit_transform(temp['target'].values.reshape(-1,1).tolist()).reshape(-1,))

train_dataset = temp.sample(frac=0.8,random_state=0)
test_dataset = temp.drop(train_dataset.index)

train_stats = temp.describe()
train_stats = train_stats.transpose()

def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)
normed_train_target=normed_train_data.pop('target')
normed_test_target=normed_test_data.pop('target')

In [15]:
from tensorboard.plugins.hparams import api as hp

In [37]:
HP_NUM_UNITS1=hp.HParam('num_units1',hp.Discrete([5,7,8]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.3))
HP_NUM_UNITS2=hp.HParam('num_units2',hp.Discrete([15,10]))
HP_LEARING_RATE=hp.HParam('learing_rate', hp.RealInterval(0.001, 0.003))

METRIC_RMSE = 'RootMeanSquaredError'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS1, HP_DROPOUT,HP_NUM_UNITS2,HP_LEARING_RATE],
    metrics=[hp.Metric(METRIC_RMSE, display_name='RMSE')]
  )

In [38]:
def train_test_model(hparams):
    model=tf.keras.Sequential()
    
    model.add(tf.keras.layers.Dense(hparams[HP_NUM_UNITS1],input_shape=(4,),name='Dense_layer_1'))
    model.add(tf.keras.layers.Dense(hparams[HP_NUM_UNITS2],name='Dense_layer_2'))
    model.add(tf.keras.layers.Dropout(hparams[HP_DROPOUT],name='dropout_layer_2'))
    model.add(tf.keras.layers.Dense(1))

    optimizer = tf.keras.optimizers.Adam(hparams[HP_LEARING_RATE])

    model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
    
    logdir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir,histogram_freq=1)
    early_stoping_callback=tf.keras.callbacks.EarlyStopping(monitor='val_loss',mode='min',verbose=1,restore_best_weights=True,patience=10)

    history=model.fit(normed_train_data,normed_train_target,validation_split=0.2,epochs=50,verbose=1,callbacks=[tensorboard_callback,early_stoping_callback],batch_size=10)
    residual=pd.DataFrame([{'y_hat':i,'y_pred':j[0],'error':(j[0]-i)} for i,j in zip(test_dataset['target'],target_standardizer.inverse_transform(model.predict(normed_test_data)))])

    return sum(residual['error'].apply(lambda x:x**2))



    

In [39]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  
    rmse = train_test_model(hparams)
    tf.summary.scalar(METRIC_RMSE, rmse, step=10)

In [40]:
!rm -rf ./logs/ 

In [48]:
session_num = 54

#Nested for loop training with all possible  combinathon of hyperparameters
for num_units1 in HP_NUM_UNITS1.domain.values:
    for num_units2 in HP_NUM_UNITS2.domain.values:
        for dropout_rate in tf.linspace(HP_DROPOUT.domain.min_value,HP_DROPOUT.domain.max_value,3):
            for learing_rate in tf.linspace(HP_LEARING_RATE.domain.min_value,HP_LEARING_RATE.domain.max_value,3):
                hparams = {
                    HP_NUM_UNITS1: num_units1,
                    HP_NUM_UNITS2: num_units2,
                    HP_DROPOUT: float("%.2f"%float(dropout_rate)), # float("%.2f"%float(dropout_rate)) limits the decimal palces to 2
                    HP_LEARING_RATE: float("%.4f"%float(learing_rate))}
                run_name = "run-%d" % session_num
                print('--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                run('logs/hparam_tuning/' + run_name, hparams)
                session_num += 1

--- Starting trial: run-54
{'num_units1': 5, 'num_units2': 10, 'dropout': 0.1, 'learing_rate': 0.001}
Epoch 1/50
32/32 [==============================] - 0s 9ms/step - loss: 0.2247 - mae: 0.3027 - mse: 0.2247 - val_loss: 0.0028 - val_mae: 0.0426 - val_mse: 0.0028
Epoch 2/50
32/32 [==============================] - 0s 3ms/step - loss: 0.1417 - mae: 0.2423 - mse: 0.1417 - val_loss: 0.0013 - val_mae: 0.0317 - val_mse: 0.0013
Epoch 3/50
32/32 [==============================] - 0s 2ms/step - loss: 0.1072 - mae: 0.1971 - mse: 0.1072 - val_loss: 0.0033 - val_mae: 0.0477 - val_mse: 0.0033
Epoch 4/50
32/32 [==============================] - 0s 2ms/step - loss: 0.1028 - mae: 0.2107 - mse: 0.1028 - val_loss: 0.0018 - val_mae: 0.0355 - val_mse: 0.0018
Epoch 5/50
32/32 [==============================] - 0s 2ms/step - loss: 0.1018 - mae: 0.2013 - mse: 0.1018 - val_loss: 5.9503e-04 - val_mae: 0.0208 - val_mse: 5.9503e-04
Epoch 6/50
32/32 [==============================] - 0s 2ms/step - loss: 0.0856 -

In [42]:
!tensorboard dev upload --logdir ./logs \
  --name "Vanilla Neuralnet with hyperparameter tuning " \
  --description "https://colab.research.google.com/drive/1L53WobdPswbsSPVBarq1o6Xth2GnvunC?usp=sharing" \
  --one_shot

2020-08-14 05:22:05.262157: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

./logs

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=373649185512-8v619h5kft38l4456nm2dj4ubeqsrvh6.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&sc

In [43]:
!tensorboard dev --logdir logs/hparam_tuning

2020-08-14 05:28:28.127302: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
usage: tensorboard dev [-h] [--helpfull] [--origin ORIGIN]
                       [--api_endpoint API_ENDPOINT]
                       [--grpc_creds_type {local,ssl,ssl_dev}]
                       [--auth_force_console]
                       {upload,update-metadata,delete,list,export,auth} ...
tensorboard dev: error: invalid choice: 'logs/hparam_tuning' (choose from 'upload', 'update-metadata', 'delete', 'list', 'export', 'auth')


In [44]:
%load_ext tensorboard

In [47]:
!tensorboard dev upload --logdir ./logs/hparam_tuning \
  --name "Vanilla Neuralnet with hyperparameter tuning " \
  --description "https://colab.research.google.com/drive/1L53WobdPswbsSPVBarq1o6Xth2GnvunC?usp=sharing" \
  --one_shot

2020-08-14 05:31:35.335952: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Data for the "graphs" plugin is now uploaded to TensorBoard.dev! Note that uploaded data is public. If you do not want to upload data for this plugin, use the "--plugins" command line argument.
Data for the "histograms" plugin is now uploaded to TensorBoard.dev! Note that uploaded data is public. If you do not want to upload data for this plugin, use the "--plugins" command line argument.
Data for the "hparams" plugin is now uploaded to TensorBoard.dev! Note that uploaded data is public. If you do not want to upload data for this plugin, use the "--plugins" command line argument.
Upload started and will continue reading any new data as it's added
to the logdir. To stop uploading, press Ctrl-C.

View your TensorBoard live at: https://tensorboard.dev/experiment/erxrXOYzTxaGpR4wgWkjYw/

[2020-08-14T05:31:38] Uploader started.
[2020-08-14T05:31:3